In [1]:
!nvidia-smi


Tue Dec 23 03:56:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# @title 1. Install Dependencies
!pip install ultralytics opencv-python-headless torch torchvision numpy pillow boto3
!pip install awscli
!pip install awscli --upgrade
!pip install awscli --upgrade --user


import os
print("✅ Installation Complete.")

/bin/bash: line 1: conda: command not found
✅ Installation Complete.


In [16]:
import torch

print("===== SYSTEM HEALTH CHECK =====")

# ---- GPU CHECK ----
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None"
print("CUDA Available:", torch.cuda.is_available())
print("GPU:", gpu_name)

print("===== CHECK COMPLETE =====")


===== SYSTEM HEALTH CHECK =====
CUDA Available: True
GPU: Tesla T4
===== CHECK COMPLETE =====


In [ ]:
import boto3
import os
import getpass
from botocore.exceptions import ClientError

def download_pilltrack_assets():
    # --- 1. Authentication & Setup ---
    print("🔐 AWS Credentials Setup:")
    AWS_ACCESS_KEY = input("Enter Access Key: ").strip()
    AWS_SECRET_KEY = getpass.getpass("Enter Secret Key: ").strip()
    AWS_REGION = input("Enter Access Key: ").strip()
    BUCKET_NAME = 'pilltrack-ml-bucket' 

    s3 = boto3.client(
        's3',
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )

    # เตรียมตัวแปรสำหรับ Return
    final_zip_path = None
    final_model_path = None

    print("\n" + "🚀" + " STARTING DOWNLOAD ".center(40, "="))
    
    # --- Task 1: ค้นหาและโหลด Zip ล่าสุด (ใช้ชื่อเดิม) ---
    try:
        print(f"📂 Scanning 'data_collection/'...")
        response = s3.list_objects_v2(Bucket=BUCKET_NAME, Prefix='data_collection/')
        
        if 'Contents' in response:
            zips = [obj for obj in response['Contents'] if obj['Key'].lower().endswith('.zip')]
            if zips:
                # หาตัวล่าสุด
                latest_zip_obj = sorted(zips, key=lambda x: x['LastModified'], reverse=True)[0]
                s3_zip_key = latest_zip_obj['Key']
                
                # ดึงชื่อไฟล์ Original ออกมา (เช่น data_v1.zip)
                original_zip_name = os.path.basename(s3_zip_key)
                final_zip_path = os.path.abspath(original_zip_name)

                print(f"   🔎 Found Latest: {original_zip_name}")
                s3.download_file(BUCKET_NAME, s3_zip_key, final_zip_path)
                print(f"   ✅ Downloaded to: {final_zip_path}")
            else:
                print("   ⚠️ No zip files found in 'data_collection/'")
        else:
            print("   ⚠️ Prefix 'data_collection/' is empty or not found.")
    except Exception as e:
        print(f"   ❌ Task 1 (Zip) Failed: {e}")

    # --- Task 2: โหลดโมเดล (ใช้ชื่อเดิม) ---
    try:
        s3_model_key = 'latest/models/seg_best_process.pt'
        original_model_name = os.path.basename(s3_model_key)
        final_model_path = os.path.abspath(original_model_name)

        print(f"\n🧠 Downloading Model: {original_model_name}...")
        s3.download_file(BUCKET_NAME, s3_model_key, final_model_path)
        print(f"   ✅ Downloaded to: {final_model_path}")
    except Exception as e:
        print(f"   ❌ Task 2 (Model) Failed: {e}")

    print("=" * 42 + "\n")
    
    return final_zip_path, final_model_path

# ---  Execution ---
zip_path, model_path = download_pilltrack_assets()

if zip_path and model_path:
    print("✨ All assets are ready!")
    print(f"📦 ZIP Path: {zip_path}")
    print(f"🤖 Model Path: {model_path}")
else:
    print("⚠️ Some downloads failed. Please check the logs above.")

🔐 AWS Credentials Setup:

🚀========== STARTING DOWNLOAD ===========
📂 Scanning 'data_collection/'...
   🔎 Found Latest: captured_20251223_091745.zip
   ✅ Downloaded to: /content/captured_20251223_091745.zip

🧠 Downloading Model: seg_best_process.pt...
   ✅ Downloaded to: /content/seg_best_process.pt

✨ All assets are ready!
📦 ZIP Path: /content/captured_20251223_091745.zip
🤖 Model Path: /content/seg_best_process.pt


### Training Phase ###

In [25]:
# @title 2. Define AI Engine Class
import torch
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
from ultralytics import YOLO

class AIEngine:
    def __init__(self, model_path, dino_size=336):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"🚀 AI Engine utilizing: {self.device}")

        # 1. YOLO Segmentation
        self.yolo = YOLO(model_path)

        # 2. DINOv2 Model
        self.dino = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14').to(self.device).eval()
        self.dino_size = dino_size

        # 3. SIFT Detector
        self.sift = cv2.SIFT_create()

        # Preprocessing
        self.preprocess = transforms.Compose([
            transforms.Resize((dino_size, dino_size), antialias=True),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

    def _enhance_image(self, img_bgr):
        lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        l2 = self.clahe.apply(l)
        lab = cv2.merge((l2, a, b))
        return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    def extract_features(self, img_bgr):
        # Enhance
        enhanced_img = self._enhance_image(img_bgr)

        # DINOv2
        img_rgb = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2RGB)
        t = self.preprocess(Image.fromarray(img_rgb)).unsqueeze(0).to(self.device)
        with torch.no_grad():
            dino_vec = self.dino(t).flatten().cpu().numpy()
        dino_vec = dino_vec / (np.linalg.norm(dino_vec) + 1e-8)

        # SIFT
        gray = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2GRAY)
        _, sift_des = self.sift.detectAndCompute(gray, None)

        return dino_vec, sift_des

    def detect_and_crop(self, img_bgr, conf=0.5, padding_pct=0.05):
        results = self.yolo(img_bgr, verbose=False, conf=conf, task='segment')
        res = results[0]
        if res.masks is not None:
            idx = res.boxes.conf.argmax()

            mask = res.masks.data[idx].cpu().numpy()
            mask = cv2.resize(mask, (img_bgr.shape[1], img_bgr.shape[0]))

            masked_img = img_bgr.copy()
            masked_img[mask == 0] = 0

            x1, y1, x2, y2 = res.boxes.xyxy[idx].cpu().numpy().astype(int)
            h, w = img_bgr.shape[:2]

            pad_w = int((x2 - x1) * padding_pct)
            pad_h = int((y2 - y1) * padding_pct)

            x1 = max(0, x1 - pad_w)
            y1 = max(0, y1 - pad_h)
            x2 = min(w, x2 + pad_w)
            y2 = min(h, y2 + pad_h)

            crop = masked_img[y1:y2, x1:x2]
            return crop if crop.size > 0 else img_bgr
        return img_bgr

print("✅ AI Engine Defined.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ AI Engine Defined.


In [26]:
import os
import shutil
import pickle
import json
import cv2
from datetime import datetime

# --- CONFIG ---
YOLO_CONF = 0.8
DINO_SIZE = 336

# 1. Setup Directories
print("📂 [1/5] Setting up workspace...")
base_dir = "pill_processing_workspace"
input_dir = os.path.join(base_dir, "input_temp")
output_db_images = os.path.join(base_dir, "database_images")

for d in [base_dir, input_dir, output_db_images]:
    if os.path.exists(d): shutil.rmtree(d)
    os.makedirs(d, exist_ok=True)

# 2. Initialize Engine
print(f"🤖 [2/5] Initializing AI Engine with: {model_path}")
engine = AIEngine(model_path, DINO_SIZE)

# 3. Unzip Images
print(f"📦 [3/5] Unzipping source: {zip_path}")
import zipfile
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(input_dir)

# 4. Processing Loop
packs_db = {}
drug_list_set = set()
saved_image_count = 0

print("\n⚡ [4/5] STARTING BATCH PROCESSING...")
tasks = []
for root, dirs, files_in_dir in os.walk(input_dir):
    folder_name = os.path.basename(root).lower()
    if "_box" in folder_name or "_blister" in folder_name:
        imgs = [os.path.join(root, f) for f in files_in_dir if f.lower().endswith(('.jpg','.png','.jpeg'))]
        if imgs: tasks.append((folder_name, imgs))

if not tasks:
    print("❌ Error: No valid folders found.")
else:
    for idx, (cls_name, img_paths) in enumerate(tasks):
        save_dir = os.path.join(output_db_images, cls_name)
        os.makedirs(save_dir, exist_ok=True)
        
        base_name = cls_name.split('_box')[0].split('_blister')[0]
        drug_list_set.add(base_name)

        for i, p in enumerate(img_paths):
            img = cv2.imread(p)
            if img is None: continue

            crop = engine.detect_and_crop(img, YOLO_CONF)
            if crop is None: continue 

            # Rotation & Feature Extraction
            for angle in [0, 90, 180, 270]:
                if angle == 0: rot = crop
                elif angle == 90: rot = cv2.rotate(crop, cv2.ROTATE_90_CLOCKWISE)
                elif angle == 180: rot = cv2.rotate(crop, cv2.ROTATE_180)
                elif angle == 270: rot = cv2.rotate(crop, cv2.ROTATE_90_COUNTERCLOCKWISE)

                dino_vec, sift_des = engine.extract_features(rot)
                key = f"{cls_name}_rot{angle}"
                if key not in packs_db: packs_db[key] = {'dino': [], 'sift': []}
                packs_db[key]['dino'].append(dino_vec)
                packs_db[key]['sift'].append(sift_des)

            # Save cropped image
            save_path = os.path.join(save_dir, f"{cls_name}_{i}.jpg")
            cv2.imwrite(save_path, crop)
            saved_image_count += 1
            
        print(f"   ✅ Processed: {cls_name} ({len(img_paths)} images)")

# 5. Save Artifacts & Logging
print("\n💾 [5/5] Saving Artifacts & Generating Logs...")

# Path Definition
pkl_path = os.path.join(base_dir, "pill_fingerprints.pkl")
json_path = os.path.join(base_dir, "drug_list.json")

# Save PKL
with open(pkl_path, 'wb') as f:
    pickle.dump(packs_db, f)

# Save JSON
json_data = {
    "drugs": sorted(list(drug_list_set)),
    "updated_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "total_drugs": len(drug_list_set),
    "total_images_processed": saved_image_count
}
with open(json_path, 'w') as f:
    json.dump(json_data, f, indent=4)

# --- SENIOR LOGGING SUMMARY ---
print("-" * 50)
print("🚀 PROCESSING COMPLETE - ARTIFACTS SUMMARY")
print("-" * 50)
print(f"📍 Database Directory : {os.path.abspath(output_db_images)}")
print(f"📍 Fingerprint File  : {os.path.abspath(pkl_path)}")
print(f"📍 Drug List (JSON)  : {os.path.abspath(json_path)}")
print(f"📊 Total Drugs       : {len(drug_list_set)}")
print(f"🖼️  Cropped Images    : {saved_image_count} files saved")
print("-" * 50)
print("✨ System is ready for the next stage!")

📂 [1/5] Setting up workspace...
🤖 [2/5] Initializing AI Engine with: /content/seg_best_process.pt
🚀 AI Engine utilizing: cuda
Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip


/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitb14_pretrain.pth


100%|██████████| 330M/330M [00:01<00:00, 224MB/s]  


📦 [3/5] Unzipping source: /content/captured_20251223_091745.zip

⚡ [4/5] STARTING BATCH PROCESSING...
   ✅ Processed: alzime_box (16 images)
   ✅ Processed: fah_blister (16 images)
   ✅ Processed: para30mg_blister (16 images)
   ✅ Processed: mypara_blister (15 images)
   ✅ Processed: para40mg_blister (16 images)
   ✅ Processed: uroflow_box (16 images)
   ✅ Processed: noxa_blister (16 images)
   ✅ Processed: maleate_blister (16 images)
   ✅ Processed: orata_box (16 images)
   ✅ Processed: turmeric_blister (16 images)
   ✅ Processed: paracap_blister (16 images)
   ✅ Processed: civoxin_box (16 images)
   ✅ Processed: sara_blister (16 images)
   ✅ Processed: decolgen_blister (16 images)
   ✅ Processed: hydroquin_box (16 images)
   ✅ Processed: bufenac_blister (16 images)
   ✅ Processed: yanhee_blister (16 images)
   ✅ Processed: gluzolyte_blister (17 images)
   ✅ Processed: flotral_box (16 images)

💾 [5/5] Saving Artifacts & Generating Logs...
----------------------------------------------

In [30]:
import os
import json

# กำหนด Path เดิมที่ใช้
base_dir = "pill_processing_workspace"
output_db_images = os.path.join(base_dir, "database_images")
pkl_path = os.path.join(base_dir, "pill_fingerprints.pkl")
json_path = os.path.join(base_dir, "drug_list.json")

print("🔍 --- VERIFYING SAVED ARTIFACTS ---")

# 1. เช็คไฟล์หลัก
print(f"\n📄 [Main Files]")
print(f"   - Fingerprints: {'✅ FOUND' if os.path.exists(pkl_path) else '❌ NOT FOUND'} -> {pkl_path}")
print(f"   - Drug List   : {'✅ FOUND' if os.path.exists(json_path) else '❌ NOT FOUND'} -> {json_path}")
print(f'Found all ✅✅✅ Ready to push to staging')
# 2. เช็คโฟลเดอร์รูปภาพ
if os.path.exists(output_db_images):
    folders = os.listdir(output_db_images)
    print(f"\n📂 [Image Folders] Found {len(folders)} drug categories:")
else:
    print("\n❌ Error: Image database folder not found!")

print("\n" + "-"*40)


🔍 --- VERIFYING SAVED ARTIFACTS ---

📄 [Main Files]
   - Fingerprints: ✅ FOUND -> pill_processing_workspace/pill_fingerprints.pkl
   - Drug List   : ✅ FOUND -> pill_processing_workspace/drug_list.json
Found all ✅✅✅ Ready to push to staging

📂 [Image Folders] Found 19 drug categories:

----------------------------------------


In [34]:
import boto3
import os
from botocore.exceptions import NoCredentialsError

# --- CONFIG S3 ---
S3_TARGET_DIR = "latest/staging_model"

print("🔐 AWS Credentials Setup:")
AWS_ACCESS_KEY = input("Enter Access Key: ").strip()
AWS_SECRET_KEY = getpass.getpass("Enter Secret Key: ").strip()
AWS_REGION = input("Enter Access Key: ").strip()
BUCKET_NAME = 'pilltrack-ml-bucket' 

# ไฟล์ที่จะ Upload
base_dir = "pill_processing_workspace"
files_to_upload = {
    os.path.join(base_dir, "pill_fingerprints.pkl"): f"{S3_TARGET_DIR}/pill_fingerprints.pkl",
    os.path.join(base_dir, "drug_list.json"): f"{S3_TARGET_DIR}/drug_list.json"
}

def upload_to_s3(local_files, bucket):
    # Initialize S3 Client
    s3 = boto3.client(
        's3',
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )
    
    print(f"☁️  Starting Upload to S3 Bucket: {bucket}\n")
    
    for local_path, s3_path in local_files.items():
        if not os.path.exists(local_path):
            print(f"❌ Skip: {local_path} (File not found!)")
            continue
            
        try:
            print(f"📤 Uploading: {os.path.basename(local_path)}...")
            # การ upload_file ของ boto3 จะทับไฟล์เดิม (Overwrite) โดย Default อยู่แล้วครับ
            s3.upload_file(local_path, bucket, s3_path)
            print(f"   ✅ Done: s3://{bucket}/{s3_path}")
        except Exception as e:
            print(f"   ❌ Failed: {str(e)}")

# Execution
upload_to_s3(files_to_upload, BUCKET_NAME)

print("\n" + "-"*50)
print("🚀 [DEPLOAYMENT COMPLETE] Files are now in Staging!")
print("-" * 50)

🔐 AWS Credentials Setup:
☁️  Starting Upload to S3 Bucket: pilltrack-ml-bucket

📤 Uploading: pill_fingerprints.pkl...
   ✅ Done: s3://pilltrack-ml-bucket/latest/staging_model/pill_fingerprints.pkl
📤 Uploading: drug_list.json...
   ✅ Done: s3://pilltrack-ml-bucket/latest/staging_model/drug_list.json

--------------------------------------------------
🚀 [DEPLOAYMENT COMPLETE] Files are now in Staging!
--------------------------------------------------
